In [1]:
# prompt: connect to google drive
#
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install matsim-tools

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import matsim
from collections import defaultdict

In [5]:
#matsim outputs
#plans_path = '/content/drive/MyDrive/ETC-DI_AP25/ABM/MATSim_outputs/berlin-v6.4.output_plans.xml.gz'
#additional data
#berlin_outline = "/content/drive/MyDrive/ETC-DI_AP25/ABM/Berlin_Bezirke_dissolve.gpkg"
#AQ raster clipped by Berlin outline
#NO2_annual_raster_Berlin = "/content/drive/MyDrive/ETC-DI_AP25/ABM/NO2_2023/NO2_2023_epsg3035_clipBerlin.tif"

base_folder = "Z:/Environment and Health/Air Quality/abm_no2/"
#matsim outputs
network_path = 'berlin_data_matsim/berlin-v6.4.output_network.xml.gz'
events_path = 'berlin_data_matsim/berlin-v6.4.output_events.xml.gz'
plans_path = 'berlin_data_matsim/berlin-v6.4.output_plans.xml.gz'

#additional data
berlin_outline = "berlin_data_shapes/Berlin_Bezirke_dissolve.gpkg"
#AQ raster clipped by Berlin outline
NO2_annual_raster_Berlin = "berlin_data_aq/NO2_2023_epsg3035_clipBerlin.tif"



# Static scenario

Outline
1. extract the **home location** of each (unique) agent
2. limit the number of agents - use only the **Berlin city-based agents** (still over 300k agents)
3. further **limit the number of agents** to selected value (e.g. to 3000)
4. get **NO2 value at the home locations** & assign the **air pollution** to the agents

#### How many agents in the scenario?

informative cell, does not need to be run everytime (as long as the input plans file does not change), the num_agents is further hardcoded (

In [6]:
# prompt: from the plans_path file can you extract how many people (Agents) are in the scenario?

# Stream through a MATSim plans file.
# The matsim.plan_reader returns a generator
plans_generator = matsim.plan_reader(base_folder+plans_path)

# To count unique persons, iterate through the generator and collect person IDs
person_ids = set() # Using a set automatically handles uniqueness

for person, plan in plans_generator:
    # The person information is in the first element of the tuple
    # The person object has attributes like 'id'
    person_id = person.attrib.get('id')
    if person_id: # Ensure the person id is not None or empty
        person_ids.add(person_id)

# Count the number of unique agents (persons)
num_agents = len(person_ids)

print(f"Number of agents in the scenario: {num_agents}")

Number of agents in the scenario: 526111


### Subset of agents for the analysis

#### limit the agents by Berlin city boundaries

In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pyproj # Required for CRS transformation
import random # For shuffling and random selection within cells
import math # For ceil in grid calculation

# Ensure pyproj is installed
try:
    import pyproj
except ImportError:
    # This might require a restart of the kernel in some environments
    print("pyproj not found, attempting to install...")
    # !pip install pyproj # Uncomment if running in an environment where pip install works directly
    import pyproj
finally:
    # Verify pyproj version if needed for specific functionalities
    # print(f"pyproj version: {pyproj.__version__}")
    pass

In [9]:
# Berlin boundary prep

# Define the assumed CRS of the MATSim coordinates
matsim_crs = 'EPSG:25832'

# Load the Berlin boundary GeoPackage and ensure a valid CRS
berlin_outline_gdf = gpd.read_file(base_folder+berlin_outline)

# --- CRS Handling for Berlin Outline ---
# Target CRS for intersection/containment checks. ETRS89-LAEA (EPSG:3035) is good for Europe-wide
# analyses and should also be suitable for your raster data.
target_crs_for_geometry_check = 'EPSG:3035'

# Check if the GeoDataFrame has a CRS
if berlin_outline_gdf.crs is None:
    print(f"Warning: Berlin outline GeoDataFrame has no CRS. Assuming {target_crs_for_geometry_check} based on previous context.")
    berlin_outline_gdf = berlin_outline_gdf.to_crs(target_crs_for_geometry_check) # Use to_crs even for initial set to ensure consistency
else:
    print(f"Original Berlin outline CRS: {berlin_outline_gdf.crs}")
    if berlin_outline_gdf.crs != target_crs_for_geometry_check:
        berlin_outline_gdf = berlin_outline_gdf.to_crs(target_crs_for_geometry_check)
        print(f"Reprojected Berlin outline CRS: {berlin_outline_gdf.crs}")

# Assuming berlin_outline_gdf contains a single polygon or a MultiPolygon representing Berlin
# Dissolve if necessary and get the single geometry for easier checking
berlin_geometry = berlin_outline_gdf.dissolve().geometry.iloc[0]

Original Berlin outline CRS: EPSG:3857
Reprojected Berlin outline CRS: EPSG:3035


In [11]:
# --- Process Plans and Collect All Eligible Home Locations (=within Berlin boundary)---

# We will collect ALL agents whose home is within Berlin, then select from them.
# if all points should be processed, the berlin_geometry.contains condition can be skipped

all_eligible_home_locations = []
processed_all_agents = set() # To ensure we only process each agent once

print(f"Collecting all eligible home locations within Berlin for spatial selection...")
# Re-initialize the plans generator to ensure we read from the beginning
plans_generator = matsim.plan_reader(base_folder+plans_path)

for person, plan in plans_generator:
    agent_id = person.attrib.get('id')

    if agent_id and agent_id not in processed_all_agents:
        processed_all_agents.add(agent_id) # Mark as processed globally for this iteration

        home_activity_found = False
        for item in plan:
            if item.tag == 'activity':
                activity_type = item.attrib.get('type')
                if activity_type and 'home' in activity_type.lower():
                    home_x_str = item.attrib.get('x')
                    home_y_str = item.attrib.get('y')

                    if home_x_str is not None and home_y_str is not None:
                        try:
                            home_x = float(home_x_str)
                            home_y = float(home_y_str)

                            # Create GeoSeries for the MATSim point
                            matsim_point_gs = gpd.GeoSeries([Point(home_x, home_y)], crs=matsim_crs)

                            # Reproject the point to the Berlin outline's CRS
                            reprojected_point_gs = matsim_point_gs.to_crs(berlin_outline_gdf.crs)
                            reprojected_point = reprojected_point_gs.iloc[0]

                            if berlin_geometry.contains(reprojected_point):
                                all_eligible_home_locations.append({
                                    'agent_id': agent_id,
                                    'x_matsim': home_x, # Original MATSim X
                                    'y_matsim': home_y, # Original MATSim Y
                                    'geometry': reprojected_point # Reprojected geometry for spatial operations
                                })
                                # Found home within Berlin, move to the next agent
                                home_activity_found = True
                                break # Stop searching activities for this person
                        except (ValueError, TypeError) as e:
                            print(f"Warning: Could not parse coordinates for agent {agent_id} home activity: {e}")
                        except Exception as e:
                            print(f"Error during CRS transformation for agent {agent_id}: {e}")
            if home_activity_found: # If a home activity was found for this person, no need to check other activities
                break

print(f"Found {len(all_eligible_home_locations)} total eligible home locations within Berlin.")
print(f"The Matsim pouints coordinates were reprojected to the {berlin_outline_gdf.crs}")

Found 376991 total eligible home locations within Berlin.
The Matsim pouints coordinates were reprojected to the EPSG:3035


In [12]:
# prompt: export the agent_homes_df to csv
agents_in_berlin_output_path = "berlin_output/agents_berlin_city.csv"

# Convert the list of dictionaries to a pandas DataFrame
all_eligible_home_locations_df = pd.DataFrame(all_eligible_home_locations)

# Now call .to_csv() on the DataFrame
all_eligible_home_locations_df.to_csv(base_folder+agents_in_berlin_output_path, index=False)
print("DataFrame exported to csv")

DataFrame exported to csv


#### limit the agents within Berlin boundaries by custom value (optional)

agents selected within grid cells (to avoid clustering)

In [13]:
from shapely.geometry import box # Import box here

# --- Spatial Selection to Avoid Clustering ---
# Strategy: Create a grid over Berlin and try to pick agents from different grid cells in a round-robin fashion.
# This helps ensure spatial distribution.

# Define grid size (adjust based on desired spread vs. number of agents)
# A 10x10 grid on Berlin's bounding box might offer 100 cells. If we need 50 agents, this is good.
# If num_agents_to_select_in_berlin is very high, consider fewer, larger cells.
num_grid_cells_x = 10
num_grid_cells_y = 10

# Get bounding box of Berlin for grid creation
minx, miny, maxx, maxy = berlin_geometry.bounds

cell_width = (maxx - minx) / num_grid_cells_x
cell_height = (maxy - miny) / num_grid_cells_y

# Create grid cells (simple rectangular grid)
grid_geometries = []
for i in range(num_grid_cells_x):
    for j in range(num_grid_cells_y):
        # Create polygon for each grid cell using shapely.geometry.box
        cell_geom = box(minx + i * cell_width, miny + j * cell_height,
                        minx + (i + 1) * cell_width, miny + (j + 1) * cell_height)
        grid_geometries.append(cell_geom)

# Create a GeoDataFrame for the grid
grid_gdf = gpd.GeoDataFrame(geometry=grid_geometries, crs=berlin_outline_gdf.crs)
grid_gdf['grid_id'] = range(len(grid_gdf))


In [14]:
num_agents_to_select_in_berlin = 3000

# Create a GeoDataFrame from all eligible home locations
all_eligible_agents_gdf = gpd.GeoDataFrame(
    all_eligible_home_locations,
    crs=berlin_outline_gdf.crs # Assign the CRS of the reprojected points ('EPSG:3035')
)

# Perform a spatial join to assign agents to grid cells
# Use 'within' predicate to ensure points are strictly inside grid cells
agents_with_grid = gpd.sjoin(all_eligible_agents_gdf, grid_gdf, how="left", predicate="within")

# Group agents by grid cell and shuffle within each group
shuffled_agents_by_grid = {}
# Use .dropna(subset=['grid_id']) in case some agents fell outside the grid bounds due to floating point errors
for grid_id, group in agents_with_grid.dropna(subset=['grid_id']).groupby('grid_id'):
    # Convert grid_id to integer as it comes from the grid_gdf index
    grid_id_int = int(grid_id)
    # Convert to list of dictionaries for easier popping
    shuffled_agents_by_grid[grid_id_int] = group.sample(frac=1, random_state=42).to_dict(orient='records')

selected_agents_list = []

# Get the list of grid_ids that actually contain agents
active_grid_ids = list(shuffled_agents_by_grid.keys())
if not active_grid_ids:
    print("No agents fell into any grid cells during spatial join. Check CRSs or grid definition.")
    # Fallback to simple random selection if grid-based fails completely
    if len(all_eligible_agents_gdf) > num_agents_to_select_in_berlin:
        agent_home_locations_gdf = all_eligible_agents_gdf.sample(n=num_agents_to_select_in_berlin, random_state=42)
    else:
        agent_home_locations_gdf = all_eligible_agents_gdf
    print(f"\nSelected {len(agent_home_locations_gdf)} agents using simple random sampling (due to grid issue).")
    print("Selected Agent Home Locations GeoDataFrame (within Berlin):")
    print(agent_home_locations_gdf.head())

else:
    # Shuffle the order of active grid_ids for a more random starting point
    random.shuffle(active_grid_ids)
    current_grid_idx = 0

    # Iterate through grid cells, selecting one agent at a time in a round-robin fashion
    # This ensures selection from different parts of the area before revisiting.
    while len(selected_agents_list) < num_agents_to_select_in_berlin and active_grid_ids:
        # Use modulo to cycle through the active grid IDs
        grid_id_to_process = active_grid_ids[current_grid_idx % len(active_grid_ids)]

        # Check if the grid_id exists in shuffled_agents_by_grid (it should if it's in active_grid_ids)
        # and if there are agents left in that cell
        if grid_id_to_process in shuffled_agents_by_grid and shuffled_agents_by_grid[grid_id_to_process]:
            # Pop one agent from this cell's shuffled list
            selected_agent_data = shuffled_agents_by_grid[grid_id_to_process].pop(0)
            selected_agents_list.append(selected_agent_data)

            # If this cell is now empty, remove it from the list of active_grid_ids
            if not shuffled_agents_by_grid[grid_id_to_process]:
                active_grid_ids.remove(grid_id_to_process)
                # No need to adjust current_grid_idx explicitly here, the modulo takes care of it
        else:
              # This case should ideally not happen if active_grid_ids is correctly managed,
              # but as a safeguard, remove the grid_id if it somehow points to an empty list
              if grid_id_to_process in active_grid_ids:
                  active_grid_ids.remove(grid_id_to_process)


        # Move to the next grid cell in the shuffled order
        if len(active_grid_ids) > 0: # Only move if there are still active grids
            current_grid_idx = (current_grid_idx + 1) % len(active_grid_ids)
        else: # All grid cells are exhausted
            break

    # Create the final GeoDataFrame of selected agents
    if selected_agents_list:
        # Separate geometry from attributes to create GeoDataFrame correctly
        # Need to be careful here: the 'geometry' key was popped earlier,
        # so we should reconstruct the DataFrame and then set the geometry column.
        # A better approach might be to keep the geometry column during sampling.
        # Let's refine the selection loop slightly.

        # Re-implementing the selection to keep geometry column
        selected_agents_with_geom = []
        # Reset generators or use a fresh list of eligible agents if available
        # Assuming `all_eligible_agents_gdf` is the full gdf of eligible agents
        # Let's retry the sampling logic using this full gdf directly.

        # A simpler approach: Group by grid_id and take the first N (or random) from each
        # cell in a loop until 50 agents are selected.

        selected_agent_ids_set = set()
        selected_agents_list = [] # List of dictionaries including geometry

        # Iterate through grid cells (shuffled order), selecting one agent per cell until target is met
        grid_ids_order = list(shuffled_agents_by_grid.keys())
        random.shuffle(grid_ids_order) # Shuffle grid order
        grid_cycle = iter(grid_ids_order * math.ceil(num_agents_to_select_in_berlin / len(grid_ids_order))) # Cycle through grids

        while len(selected_agent_ids_set) < num_agents_to_select_in_berlin and shuffled_agents_by_grid:
            try:
                current_grid_id = next(grid_cycle)
            except StopIteration:
                print("Ran out of grid cells with agents before selecting enough agents.")
                break # Stop if we've cycled through all grids with agents

            if current_grid_id in shuffled_agents_by_grid and shuffled_agents_by_grid[current_grid_id]:
                # Pop the next agent from this cell
                agent_data = shuffled_agents_by_grid[current_grid_id].pop(0)
                # Ensure agent_id is added to the set to track uniqueness
                agent_id = agent_data['agent_id']
                if agent_id not in selected_agent_ids_set:
                    selected_agents_list.append(agent_data)
                    selected_agent_ids_set.add(agent_id)

            # If the list for this grid is now empty, remove the grid from consideration
            if current_grid_id in shuffled_agents_by_grid and not shuffled_agents_by_grid[current_grid_id]:
                del shuffled_agents_by_grid[current_grid_id]


        if selected_agents_list:
              # Create the final GeoDataFrame from the collected list
              # Ensure the geometry column is correctly handled during GeoDataFrame creation
              agent_home_locations_gdf = gpd.GeoDataFrame(selected_agents_list, crs=berlin_outline_gdf.crs)
              print(f"\nSelected {len(agent_home_locations_gdf)} agents with spatially distributed home locations within Berlin.")
              print("Selected Agent Home Locations GeoDataFrame (within Berlin):")
              print(agent_home_locations_gdf.head())
        else:
              print("\nNo agents selected after spatial sampling logic. Check if any agents fell within the grid.")
              # Fallback if grid sampling somehow yields no results
              if len(all_eligible_agents_gdf) > num_agents_to_select_in_berlin:
                agent_home_locations_gdf = all_eligible_agents_gdf.sample(n=num_agents_to_select_in_berlin, random_state=42)
              else:
                agent_home_locations_gdf = all_eligible_agents_gdf
              print(f"\nSelected {len(agent_home_locations_gdf)} agents using simple random sampling (due to grid sampling issue).")
              print("Selected Agent Home Locations GeoDataFrame (within Berlin):")
              print(agent_home_locations_gdf.head())

# Now agent_home_locations_gdf contains home locations of `num_agents_to_select_in_berlin` (or fewer if not available)
# agents within Berlin, selected to be spatially distributed,
# and its CRS matches that of berlin_outline_gdf (EPSG:3035).
# You can proceed with spatial operations like joining with the NO2 raster.

Ran out of grid cells with agents before selecting enough agents.

Selected 2847 agents with spatially distributed home locations within Berlin.
Selected Agent Home Locations GeoDataFrame (within Berlin):
          agent_id       x_matsim      y_matsim  \
0  berlin_2a8ee486  783799.985857  5.824417e+06   
1  berlin_816c5472  788869.771966  5.840491e+06   
2  berlin_4f582033  802830.100512  5.831748e+06   
3  berlin_4f389be6  786756.072566  5.838889e+06   
4      bb_a7d85154  807365.300000  5.836209e+06   

                          geometry  index_right  grid_id  
0  POINT (4536986.598 3269901.216)           14       14  
1  POINT (4542277.219 3285893.081)           28       28  
2  POINT (4556102.395 3276956.337)           56       56  
3   POINT (4540142.82 3284322.454)           18       18  
4  POINT (4560695.642 3281349.743)           67       67  


In [15]:
# prompt: export the agent_homes_df to csv
agents_in_berlin_selection_output_path = "berlin_output/agents_berlin_city_selected3000.csv"

# Now call .to_csv() on the DataFrame
agent_home_locations_gdf.to_csv(base_folder+agents_in_berlin_selection_output_path, index=False)
print("DataFrame exported to csv")

DataFrame exported to csv


### Get NO2 value at the home locations

In [16]:
try:
    import rasterio
except ImportError:
    !pip install rasterio
    import rasterio

In [17]:
# Define the CRS of the MATSim coordinates (ETRS89 / UTM zone 33N for Berlin)
matsim_crs = 'EPSG:25832'
# Define the target CRS for spatial operations (ETRS89-LAEA for Europe, matches the raster)
target_crs_for_raster = 'EPSG:3035'

#process all agents within Berlin (without selection a subset)
agent_home_locations_gdf = all_eligible_agents_gdf

# Load the NO2 raster
try:
    with rasterio.open(base_folder+NO2_annual_raster_Berlin) as src:
        raster_crs = src.crs
        print(f"NO2 Raster CRS: {raster_crs}")

        # Read the first band and transform
        no2_band = src.read(1)
        raster_transform = src.transform

        # Check if raster CRS matches the target CRS, reproject agent points if necessary
        if raster_crs != target_crs_for_raster:
            print(f"Warning: Raster CRS ({raster_crs}) does not match target CRS ({target_crs_for_raster}). Reprojecting agent home locations.")
            # Reproject the agent home locations GeoDataFrame to match the raster CRS
            # Assuming agent_home_locations_gdf is already loaded and has a CRS set
            # (it should have EPSG:3035 from the previous step, but reprojecting is safer)
            if agent_home_locations_gdf.crs is None:
                print("Error: agent_home_locations_gdf has no CRS. Cannot reproject.")
                # Handle this error: e.g., set CRS or skip raster sampling
            elif agent_home_locations_gdf.crs != raster_crs:
                agent_home_locations_gdf = agent_home_locations_gdf.to_crs(raster_crs)
                print(f"Agent Home Locations GeoDataFrame reprojected to Raster CRS: {agent_home_locations_gdf.crs}")
            else:
                 print("Agent Home Locations GeoDataFrame CRS matches Raster CRS.")

        # Check if agent_home_locations_gdf is empty or missing required columns
        if agent_home_locations_gdf.empty or 'geometry' not in agent_home_locations_gdf.columns:
             print("Agent home locations GeoDataFrame is empty or missing 'geometry' column. Cannot sample raster.")
             # Add a placeholder column with None values if the dataframe exists but is empty
             if 'NO2_value' not in agent_home_locations_gdf.columns:
                  agent_home_locations_gdf['NO2_value'] = None
        else:
            # Function to get raster value at a given point geometry
            # This function uses the rasterio's sample method which handles coordinates directly
            def get_raster_value_at_point(point_geometry, raster_source):
                # rasterio.sample takes a list of point tuples [(x, y), ...]
                # The point_geometry is a shapely Point, get its coordinates
                if point_geometry is None or point_geometry.is_empty:
                    return None
                try:
                    # Use src.sample directly with the point geometry.
                    # src.sample expects points in the source CRS, which the reprojected
                    # agent_home_locations_gdf['geometry'] should now match.
                    value_generator = raster_source.sample([(point_geometry.x, point_geometry.y)])
                    # src.sample returns an iterable, usually with one item for one point
                    value = next(value_generator)[0] # Get the value from the array (band 1)
                    # Handle potential NoData values if specified in the raster
                    if src.nodata is not None and value == src.nodata:
                         return None # Treat NoData as None
                    return value
                except Exception as e:
                    print(f"Error sampling raster at point ({point_geometry.x}, {point_geometry.y}): {e}")
                    return None

            # Apply the function to each row in the agent_home_locations_gdf
            # Make sure the GeoDataFrame has a 'geometry' column with Point objects in the raster CRS
            agent_home_locations_gdf['NO2_value'] = agent_home_locations_gdf['geometry'].apply(
                 lambda geom: get_raster_value_at_point(geom, src)
            )

            print("\nNO2 values sampled at agent home locations:")
            print(agent_home_locations_gdf[['agent_id', 'x_matsim', 'y_matsim', 'NO2_value']].head())

            # Optional: Show summary statistics for the sampled NO2 values
            print("\nSummary statistics for sampled NO2 values:")
            print(agent_home_locations_gdf['NO2_value'].describe())


except rasterio.errors.RasterioIOError as e:
    print(f"Error opening NO2 raster file: {e}")
    print("Cannot proceed with raster sampling.")
    # Ensure the NO2_value column exists even if the raster fails to load
    if 'NO2_value' not in agent_home_locations_gdf.columns:
         agent_home_locations_gdf['NO2_value'] = None
except Exception as e:
    print(f"An unexpected error occurred during raster processing: {e}")
    # Ensure the NO2_value column exists even if other errors occur
    if 'NO2_value' not in agent_home_locations_gdf.columns:
         agent_home_locations_gdf['NO2_value'] = None


print("The GeoDataFrame `agent_home_locations_gdf` now contains the 'NO2_value' assigned to each selected agent.")

NO2 Raster CRS: EPSG:3035

NO2 values sampled at agent home locations:
      agent_id   x_matsim    y_matsim  NO2_value
0  bb_00afd203  811364.91  5814352.60   8.339029
1  bb_01957e69  780852.22  5812441.37       -inf
2  bb_01ccc469  792574.51  5815250.61       -inf
3  bb_02a9b568  810986.06  5831918.25       -inf
4  bb_02fdaa13  810738.80  5814317.68  10.251132

Summary statistics for sampled NO2 values:
count    3.769910e+05
mean             -inf
std               NaN
min              -inf
25%      1.332574e+01
50%      1.567789e+01
75%      1.843211e+01
max      2.154372e+01
Name: NO2_value, dtype: float64
The GeoDataFrame `agent_home_locations_gdf` now contains the 'NO2_value' assigned to each selected agent.


In [18]:
import numpy as np

# Replace -inf values in the 'NO2_value' column with NaN
# -inf values often appear in raster sampling when the point falls exactly on a pixel boundary or due to interpolation issues at edges/nodata areas.

# Check if the column exists and is numeric
if 'NO2_value' in agent_home_locations_gdf.columns and pd.api.types.is_numeric_dtype(agent_home_locations_gdf['NO2_value']):
    print("Replacing -inf NO2_value with NaN...")
    initial_neg_inf_count = (agent_home_locations_gdf['NO2_value'] == -np.inf).sum()

    # Use replace method or direct boolean indexing
    # Using replace is often cleaner
    agent_home_locations_gdf['NO2_value'] = agent_home_locations_gdf['NO2_value'].replace(-np.inf, np.nan)

    # Alternatively, using boolean indexing:
    # agent_home_locations_gdf.loc[agent_home_locations_gdf['NO2_value'] == -np.inf, 'NO2_value'] = np.nan

    final_nan_count = agent_home_locations_gdf['NO2_value'].isna().sum()
    print(f"Initial -inf count: {initial_neg_inf_count}")
    print(f"Final NaN count (includes original NaNs and replaced -inf): {final_nan_count}")
else:
    print("'NO2_value' column not found or is not numeric in agent_home_locations_gdf. Skipping -inf replacement.")

# Verify the changes
print("\nAgent Home Locations GeoDataFrame after replacing -inf with NaN:")
print(agent_home_locations_gdf[['agent_id', 'NO2_value']].head())

# Optional: Check descriptive statistics again
print("\nSummary statistics after replacement:")
print(agent_home_locations_gdf['NO2_value'].describe())


Replacing -inf NO2_value with NaN...
Initial -inf count: 3035
Final NaN count (includes original NaNs and replaced -inf): 3035

Agent Home Locations GeoDataFrame after replacing -inf with NaN:
      agent_id  NO2_value
0  bb_00afd203   8.339029
1  bb_01957e69        NaN
2  bb_01ccc469        NaN
3  bb_02a9b568        NaN
4  bb_02fdaa13  10.251132

Summary statistics after replacement:
count    373956.000000
mean         15.736704
std           3.167824
min           2.357919
25%          13.381557
50%          15.694091
75%          18.448425
max          21.543722
Name: NO2_value, dtype: float64


In [19]:
# Now agent_home_locations_gdf has a 'NO2_value' column
# Save the GeoDataFrame with the new NO2 column if needed
agents_with_no2_output_path = "berlin_output/agents_berlin_city_with_NO2_v2.csv"
# Save as CSV, geometry will be converted to WKT by default (or just drop it if not needed)
agent_home_locations_gdf.to_csv(base_folder+agents_with_no2_output_path, index=False)
print(f"Agent data with NO2 values saved to {agents_with_no2_output_path}")

Agent data with NO2 values saved to berlin_output/agents_berlin_city_with_NO2_v2.csv
